In [25]:
from asammdf import MDF
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
from pprint import pprint
import sys
import logging
import time
import numpy as np


sys.path.append(str(Path("D:/utils/grafana-log-viewer").resolve()))

from decoder.D65.send_d65_data import *

logging.basicConfig(level=logging.INFO, format='%(message)s')

start_date= datetime(2025, 11, 13, tzinfo=ZoneInfo("America/Vancouver"))
end_date = start_date + timedelta(days=1)
files: list[CSVContent] = [f for f in get_all_d65_cancloud_files(start=start_date, end=end_date) if f[1] == 'Lower']

print(f"Total files to process: {len(files)}")

[2025-11-17 12:26:07,742] send_d65_data.py:609  📁 Reading CANCloud files from D:\d65files ...
[2025-11-17 12:26:08,243] send_d65_data.py:617  ✔️  [CANCloud] Found 6 files in 0.499s


Total files to process: 3


In [47]:
def process(f: Path, pgn: int) -> tuple[bool, Path]:
    single_mdf = MDF(f, process_bus_logging=False)
    df = single_mdf.to_dataframe()
    df.columns = [col.replace("CAN_DataFrame.", "") for col in df.columns]
    def check_column(col_name: str) -> bool:
        if col_name not in df.columns:
            return False

        df[col_name + "hex"] = df[col_name].apply(lambda x: hex(x))
        ids = df[col_name + "hex"].unique().tolist()
        ids.sort()
        return (pgn << 8) in [int(_id,16) & 0x00FFFF00 for _id in ids]
    
    for col in ["ID"] + [c for c in df.columns if c.startswith("ID_")]:
        if check_column(col):
            return (True, f)

    return (False, f)

test_pgn = 0xee0f
ts = time.time()
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process, f, test_pgn) for f, _, _ in files]
    _found_list = []
    last_list_size = 0
    for i, future in enumerate(as_completed(futures)):
        found, _file = future.result()
        new_ts = time.time()
        if new_ts - ts > 5:
            ts = new_ts
            if last_list_size != len(_found_list):
                last_list_size = len(_found_list)
                logging.info(f"Progress: {i}/{len(files)} files processed, {_found_list} found so far.")
            else:
                logging.info(f"Progress: {i}/{len(files)} files processed.")
        if found:
            logging.info(f"[{i:03d}] ✅ Found in {shortpath(_file)}")
            _found_list.append(_file)

logging.info(f"Total files containing 0x{test_pgn:08X}: {len(_found_list)}")


[2025-11-17 13:37:39,877] 2513669529.py:37 [000] ✅ Found in D:/d65files/5A72CE4C_00001501_00000001-6916290E.MF4
[2025-11-17 13:37:42,142] 2513669529.py:37 [002] ✅ Found in D:/d65files/5A72CE4C_00001502_00000001-69174F07.MF4
[2025-11-17 13:37:42,143] 2513669529.py:40 Total files containing 0x0000EE0F: 2


In [48]:
dbc_files = get_d65_dbc_files()['Lower']
dbc_main = [dbc for dbc in dbc_files if 'Main' in dbc.name][0]
mdf = MDF().stack(files=[f for f in _found_list], process_bus_logging=False)
decoded = mdf.extract_bus_logging(database_files={"CAN": [(dbc_main, 0)]})

[2025-11-17 13:37:53,269] canmatrix.py:452 Value 0 is not valid for DCDC1_Output_current_limit. Min=100 and Max=149
[2025-11-17 13:37:53,270] canmatrix.py:452 Value 0 is not valid for DCDC1_Output_voltage_OV_fault. Min=22.05 and Max=36
[2025-11-17 13:37:53,270] canmatrix.py:452 Value 0 is not valid for DCDC1_Temp_coolant_warning. Min=40 and Max=100
[2025-11-17 13:37:53,271] canmatrix.py:452 Value 0 is not valid for DCDC1_Output_Voltage_setpoint. Min=22 and Max=32
[2025-11-17 13:37:53,271] canmatrix.py:452 Value 0 is not valid for DCDC1_Output_voltage_UV_fault. Min=20 and Max=31.95
[2025-11-17 13:37:53,272] canmatrix.py:452 Value 0 is not valid for DCDC1_Temperature_coolant_fault. Min=50 and Max=110
[2025-11-17 13:37:53,272] canmatrix.py:452 Value 0 is not valid for DCDC2_Output_current_limit. Min=100 and Max=149
[2025-11-17 13:37:53,273] canmatrix.py:452 Value 0 is not valid for DCDC2_Output_voltage_OV_fault. Min=22.05 and Max=36
[2025-11-17 13:37:53,274] canmatrix.py:452 Value 0 is no

In [49]:
df = decoded.to_dataframe()

In [50]:
channels = [ch for ch in mdf.iter_channels() if len(ch.timestamps) > 0]
unique_ids = [[_id for _id in np.unique(ch.samples['CAN_DataFrame.ID']) if (_id >> 8) & 0xFFFF == 0xEE0F] for ch in channels]

In [51]:
hex(unique_ids[0][0])

'0x1fee0fbe'

TypeError: apply_along_axis() got multiple values for argument 'axis'